# 0. Importación de librerias

In [1]:
# Librerías para el análisis de datos
import numpy as np  # Biblioteca para operaciones numéricas, especialmente útil para trabajar con matrices y arrays
import pandas as pd  # Biblioteca para manipulación y análisis de datos, especialmente para DataFrames
import statsmodels.api as sm  # Biblioteca para realizar modelos estadísticos, útil para regresiones y otros análisis estadísticos
from collections import Counter  # Contador de elementos en colecciones, útil para contar la frecuencia de elementos en una lista o array

# Librería para optimización
from gurobipy import Model, GRB, quicksum  # Gurobi es una biblioteca de optimización matemática. 
# Model permite definir el modelo de optimización,
# GRB contiene constantes (como tipos de variables y sentido de optimización),
# quicksum permite realizar sumas de forma rápida y eficiente en Gurobi.

# Librerías operacionales
import os  # Biblioteca para interactuar con el sistema operativo (ej. manejo de archivos y rutas)
from datetime import datetime  # Módulo para trabajar con fechas y tiempos, útil para capturar fechas actuales o manipular datos de tiempo


# <h2>1. Carga de datos desde excel a dataframes de pandas</h2>


En primer lugar se cargan los datos iniciales desde los csv que nos proveen. (Esto puede tardar un ratito)

In [2]:
url_tratamiento = 'https://drive.usercontent.google.com/download?id=1KTRwYGaWoQQnZwbk8VpdxKULMaOReoF5&authuser=0&confirm=t&uuid=4e4d7983-c5a5-412d-9f5b-b9bda1068b73&at=AENtkXZSxkDr84kWrDlz6ANq4ov2%3A1730951312566'

df_tratamiento = pd.read_csv("Tratamiento.csv")

In [3]:
file_id = '173bRZQG7NWfdpHJ-o4-NA3ieH1-Fhyok'
url_informacion_de_clientes = f'https://drive.google.com/uc?id={file_id}'

df_informacion_de_clientes = pd.read_csv("Informacion_Clientes.csv")

In [4]:
url_simulaciones_clientes = 'https://drive.usercontent.google.com/download?id=1IXyKwtKFLCUsAV1MtNqktiwKaHzV2D5A&authuser=0&confirm=t&uuid=edd22376-238f-4c1f-b603-728b07bafd7f&at=AENtkXaEURpV52p_BdWxyisvjhSQ%3A1730951026384'

df_simulaciones_clientes = pd.read_csv("Simulaciones_clientes.csv")

In [5]:
file_id = '1Z4jMzZeD2q-4-ioSgyimppAdLZzqDkt3'
url_ventas = f'https://drive.google.com/uc?id={file_id}'

df_ventas = pd.read_csv("Ventas.csv")

## <h3>1.1 Carga de 'Informacion_Clientes.csv'</h3>

En primer lugar se cargará la información de los clientes. Esto incluye las siguientes características de los clientes:

* **unnamed**: algo como uid
* **Rut**: identificador de Chile (supongo que por privacidad va desde 0 a max de observaciones)
* **Género**: Masculino o femenino
* **Categoría_Digital**: Si el cliente es digital o no
* **Elasticidad_Precios**: Baja, media o alta
* **Nacionalidad**: Chileno o extranjero
* **Propensión**: Número entre 0 y 1 que idica que tan propenso a cursar un credito es el cliente
* **Probabilidad_No_Pago**: Número entre 0 y 1 que indica la probabilidad de que el cliente no pague la deuda
* **Edad**: Numero entero de edad en años
* **Renta**: Renta promedio de los últimos 12 meses
* **Oferta_Consumo**: Monto máximo que puede cursar un cliente dado sus antecedentes crediticios y situación socioeconómica. 
* **Deuda_CMF**: Deuda que tiene el cliente en otros bancos. Efectivamente es deuda pendiente, pero de créditos otorgados por la competencia.
* **Tiempo_como_cliente**: Número de tiempo(no sé en que medida está) que el cliente lleva en el banco


Se elimina el tiempo como cliente ya que no aporta información

In [6]:
df_informacion_de_clientes.drop(columns=['Tiempo_como_cliente'], inplace=True)

## <h3>1.2 Carga de 'Simulaciones_Clientes.csv'</h3>



En segundo lugar se cargaran las simulaciones hechas por los clientes en la página del banco. Esto incluye las siguientes características de las simulaciones:
* **unnamed**: Supongo que es el número de simulacion registrada, un tipo de identificador de la simulación
* **fecha**: yyyy-mm-dd fecha de la simulación
* **rut**: identificador de Chile del cliente que hizo la simulacion
* **monto_simulado**: monto prestado al cliente
* **plazo_simulado**: plazo en **meses** del crédito
* **tasa_simulado**: costo para el cliente del credito

In [7]:
df_simulaciones_clientes = df_simulaciones_clientes[df_simulaciones_clientes['Monto_Simulado'] > 0]

## <h3>1.3 Carga de 'Tratamiento.csv'</h3>

En tercer lugar se cargara el tratamiento que ha tenido el banco con el cliente, es decir, cómo se han contactado con él. Esto incluye las siguientes características:

* **unnamed**: Número de tratamiento registrado
* **fecha**: yyyy-mm-dd
* **rut**: Identificador de Chile del cliente con el que se tiene el tipo de trato
* **n_correos**: Cantidad de correos que se enviaron en el mes que sale la fecha. Es decir, si sele fecha '2024-03-01', correspondería a los correos enviados en marzo de 2024.
* **asg_ejec**: Si el cliente tiene un ejecutivo asignado

## <h3>1.4 Carga de 'Ventas.csv'</h3>

Por último se cargaran las ventas que ha tenido el banco con el cliente. Esto incluye las siguientes características:

* **unnamed**: Índice sin significado
* **fecha**: yyyy-mm-dd -> fecha en la que se concretó la venta
* **rut**: identificador de Chile del cliente al que se le concretó la venta

# <h2>2. Joints de datos<h2>

In [8]:
# Unir los DataFrames 'df_informacion_de_clientes' y 'df_simulaciones_clientes' en base a la columna 'rut'
# El método 'how="left"' asegura que todos los registros de 'df_informacion_de_clientes' se conserven,
# incluso si no tienen coincidencia en 'df_simulaciones_clientes'.
df_simulaciones_e_informacion_de_clientes = pd.merge(
    df_informacion_de_clientes, 
    df_simulaciones_clientes, 
    on='rut', 
    how='left'
)

# Crear una nueva columna 'simulo' que indica si el cliente tiene un 'Monto_Simulado' o no
# El método 'notna()' devuelve True para valores no nulos y False para nulos.
# Luego, 'astype(int)' convierte estos valores booleanos en enteros (1 para True, 0 para False).
df_simulaciones_e_informacion_de_clientes['simulo'] = df_simulaciones_e_informacion_de_clientes['Monto_Simulado'].notna().astype(int)

# Eliminar columnas innecesarias 'Unnamed: 0_x' y 'Unnamed: 0_y' que podrían haber surgido durante la carga o manipulación de datos
df_simulaciones_e_informacion_de_clientes.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)


In [9]:
# Unir los DataFrames 'df_simulaciones_e_informacion_de_clientes' y 'df_ventas' en base a las columnas 'rut' y 'fecha'
# El método 'how="left"' asegura que todos los registros de 'df_simulaciones_e_informacion_de_clientes' se conserven,
# incluso si no tienen coincidencia en 'df_ventas'.
df_simulaciones_e_informacion_de_clientes_ventas = pd.merge( 
    df_simulaciones_e_informacion_de_clientes, 
    df_ventas, 
    on=['rut', 'fecha'], 
    how='left'
)

# Crear una nueva columna 'venta' que indica si existe una venta asociada al cliente y la fecha específica
# El método 'notna()' verifica si hay un valor no nulo en la columna 'Unnamed: 0' (que indica presencia de una venta)
# Luego, 'astype(int)' convierte estos valores booleanos en enteros (1 para True, 0 para False).
df_simulaciones_e_informacion_de_clientes_ventas['venta'] = df_simulaciones_e_informacion_de_clientes_ventas['Unnamed: 0'].notna().astype(int)


In [10]:
# Unir los DataFrames 'df_simulaciones_e_informacion_de_clientes_ventas' y 'df_tratamiento' en base a las columnas 'rut' y 'fecha'
# La unión se realiza con 'how="left"' para conservar todos los registros de 'df_simulaciones_e_informacion_de_clientes_ventas'
# incluso si no tienen coincidencia en 'df_tratamiento'.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento = pd.merge( 
    df_simulaciones_e_informacion_de_clientes_ventas, 
    df_tratamiento, 
    on=['rut', 'fecha'], 
    how='left'
)

# Renombrar la columna 'Unnamed: 0_x' a 'venta' para tener una denominación clara y comprensible.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento.rename(columns={'Unnamed: 0_x': 'venta'}, inplace=True)

# Actualizar la columna 'venta' para que indique si el registro de venta no es nulo (1 si hay venta, 0 si no hay)
# Se utiliza 'notna()' para verificar la presencia de un valor y 'astype(int)' para convertir el resultado booleano a entero.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['venta'] = df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['venta'].notna().astype(int)

# Crear una nueva columna 'mes' que extrae el mes y año de la columna 'fecha'
# Primero se convierte 'fecha' al formato datetime, luego 'dt.to_period('M')' obtiene el periodo del mes/año.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['mes'] = pd.to_datetime(df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['fecha']).dt.to_period('M')


# 3. CLUSTERING POR POLITICAS

### Seteo de cluster. Aquí se definen las variables y sus cortes. La idea es que el algoritmo de RL haga sus acciones en esta sección

Aquí se definen que variables se utilizarán para crear el cluster

In [11]:
df_informacion_de_clientes_procesados_cluster_definitivo = df_informacion_de_clientes[['rut', 'Renta', 'Elasticidad_Precios', 'Probabilidad_No_Pago']].copy()


Aqui se definen en que partes y en cuantas partes se particionarán las variables escogidas anteriormente

In [12]:
# Crear una copia del DataFrame 'df_informacion_de_clientes_procesados_cluster_definitivo' para trabajar sin modificar el original
df = df_informacion_de_clientes_procesados_cluster_definitivo.copy()


# Ordenar el DataFrame por 'Probabilidad_No_Pago' en orden ascendente
df_sorted = df.sort_values(by='Probabilidad_No_Pago')

# Seleccionar los primeros 205000 clientes con la menor 'Probabilidad_No_Pago'
mejores_pagadores = df_sorted.head(205000)

# Añadir una columna 'Grupo_Pago' al DataFrame original, inicializando con 'Otros'
df['Categoria_Probabilidad_No_Pago'] = 'Malos pagadores'

# Asignar la etiqueta 'Mejores Pagadores' a los clientes seleccionados
df.loc[mejores_pagadores.index, 'Categoria_Probabilidad_No_Pago'] = 'Mejores Pagadores'


# Ordenar el DataFrame por 'Renta' en orden ascendente
df_sorted = df.sort_values(by='Renta')

# Seleccionar los primeros 205000 clientes con la menor 'Probabilidad_No_Pago'
mejores_rentas = df_sorted.head(205000)

# Añadir una columna 'Grupo_Pago' al DataFrame original, inicializando con 'Otros'
df['Categoria_Renta'] = 'Rentas bajas'

# Asignar la etiqueta 'Mejores Pagadores' a los clientes seleccionados
df.loc[mejores_rentas.index, 'Categoria_Renta'] = 'Mejores Pagadores'

df


,rut,Renta,Elasticidad_Precios,Probabilidad_No_Pago,Categoria_Probabilidad_No_Pago,Categoria_Renta
0,1,6.258183e+05,Alta,0.028445,Malos pagadores,Mejores Pagadores
1,2,3.172616e+05,Baja,0.014320,Malos pagadores,Mejores Pagadores
2,3,1.240551e+07,Baja,0.002156,Mejores Pagadores,Rentas bajas
3,4,5.441466e+05,Alta,0.034418,Malos pagadores,Mejores Pagadores
4,5,1.870225e+05,Media,0.014978,Malos pagadores,Mejores Pagadores
...,...,...,...,...,...,...
543646,543647,1.176598e+05,Baja,0.037291,Malos pagadores,Mejores Pagadores
543647,543648,1.558612e+06,Baja,0.035877,Malos pagadores,Rentas bajas
543648,543649,9.449508e+05,Media,0.023306,Malos pagadores,Rentas bajas
543649,543650,1.039964e+06,Media,0.015121,Malos pagadores,Rentas bajas


In [13]:
# Concatenar las variables especificadas en una nueva columna 'categoria_clusterizacion'
# La columna resultante combinará varias categorías en una descripción detallada del perfil del cliente.
# Convertimos cada columna a tipo string para asegurarnos de que los datos sean compatibles para la concatenación.

df['categoria_clusterizacion'] = ('Cliente con elasticidad' +
    df['Elasticidad_Precios'].astype(str) + ' que es ' +              # Categoría de elasticidad del cliente
    df['Categoria_Probabilidad_No_Pago'].astype(str) + ' con una ' +     # Categoría de propensión, seguida de "con una"
    df['Categoria_Renta'].astype(str)                        # Categoría de renta
)


In [14]:
# Asignar un número único a cada entrada distinta en la columna 'categoria_clusterizacion'
# Se convierte la columna a tipo 'category', lo cual facilita la asignación de códigos numéricos únicos.
# 'cat.codes' asigna un código numérico único para cada valor único de 'categoria_clusterizacion'.
df['categoria_clusterizacion_numerica'] = df['categoria_clusterizacion'].astype('category').cat.codes


In [15]:
# Crear una copia del DataFrame con solo las columnas 'rut', 'categoria_clusterizacion' y 'categoria_clusterizacion_numerica'
# Esta copia se almacena en el nuevo DataFrame 'asignacion_clusters', el cual contendrá únicamente la identificación del cliente (rut),
# la descripción del perfil ('categoria_clusterizacion') y el código numérico asignado a cada perfil ('categoria_clusterizacion_numerica').
asignacion_clusters = df[['rut', 'categoria_clusterizacion', 'categoria_clusterizacion_numerica']].copy()


# 4. Estimacion de curvas de elasticidad por cluster

In [20]:
# Realizar una unión entre 'df_simulaciones_e_informacion_de_clientes_ventas' y 'asignacion_clusters' usando la columna 'rut' como clave
# Esta unión ('merge') se realiza con 'how="left"', lo que asegura que todos los registros de 'df_simulaciones_e_informacion_de_clientes_ventas' 
# se conserven, incluyendo aquellos sin coincidencia en 'asignacion_clusters'.
# La finalidad es agregar la información de clusterización (categoría y código numérico) al DataFrame de simulaciones y ventas.
df_estimar_elasticidad = pd.merge(df_simulaciones_e_informacion_de_clientes_ventas, asignacion_clusters, on='rut', how='left')


### Este código realiza un análisis de elasticidad de ingresos en función de clusters de clientes. Primero, agrupa los datos por clusters definidos a través de variables de segmentación y filtra solo los datos relevantes para cada cluster. Luego, para cada cluster, se ajusta un modelo de regresión logística para predecir la probabilidad de aceptación de una simulación de crédito en función de la tasa de interés. A partir de este modelo, se crea una cuadrícula de tasas para estimar la probabilidad de aceptación y calcular el revenue potencial de cada simulación, teniendo en cuenta el monto medio simulado, el plazo medio simulado y la probabilidad media de no pago del cluster. Posteriormente, se determina la tasa que maximiza el revenue esperado y se calcula el número esperado de créditos aceptados, junto con el número de clientes únicos en cada cluster. Finalmente, los resultados se agregan tanto en listas globales como en un nuevo DataFrame, y luego se integran en el DataFrame original df_estimar_elasticidad, lo que permite analizar el revenue esperado total y otros indicadores clave en cada cluster.

In [21]:
# Inicializar listas para almacenar resultados globales de revenue, clientes, créditos y simulaciones
lista_revenue = []
lista_clientes = []
lista_creditos = []
lista_simulaciones = []

cluster_results = []  # Lista para almacenar resultados específicos de cada cluster

# Obtener los números únicos de cada cluster
cluster_numbers = df_estimar_elasticidad['categoria_clusterizacion_numerica'].unique()

# Iterar sobre cada cluster identificado por 'categoria_clusterizacion_numerica'
for cluster_num in cluster_numbers:
    # Filtrar los datos correspondientes al cluster actual
    df_cluster = df_estimar_elasticidad[df_estimar_elasticidad['categoria_clusterizacion_numerica'] == cluster_num]
    
    # Asegurarse de que existen datos para ambos casos: venta == 1 y venta == 0
    if df_cluster.empty or df_cluster['venta'].isnull().all():
        continue  # Saltar este cluster si no cumple con la condición
    
    # Remover filas donde 'venta' o 'Tasa_Simulado' son nulos o infinitos
    df_cluster = df_cluster.replace([np.inf, -np.inf], np.nan)
    df_cluster = df_cluster.dropna(subset=['venta', 'Tasa_Simulado', 'Plazo_Simulado', 'Monto_Simulado', 'Probabilidad_No_Pago'])
    
    # Saltar el cluster si no hay suficientes puntos de datos
    if df_cluster.shape[0] < 10:
        continue
    
    # Extraer las variables 'venta' (como variable dependiente) y 'Tasa_Simulado' (como predictor)
    y = df_cluster['venta']
    X = df_cluster[['Tasa_Simulado']]
    
    # Añadir un término constante para el intercepto
    X = sm.add_constant(X)
    
    # Remover filas con valores NaN o Inf en X o y
    is_finite = np.isfinite(X).all(1) & np.isfinite(y)
    X = X[is_finite]
    y = y[is_finite]
    
    # Asegurarse de que después de remover NaN/Inf, todavía hay suficientes datos
    if len(y) < 10:
        continue
    
    # Ajustar el modelo de regresión logística
    logit_model = sm.Logit(y, X)
    try:
        result = logit_model.fit(disp=0)
    except:
        continue  # Saltar el cluster si el modelo no converge
    
    # Crear una cuadrícula de valores de 'Tasa_Simulado' para predicciones
    tasa_min = df_cluster['Tasa_Simulado'].min()
    tasa_max = df_cluster['Tasa_Simulado'].max()
    tasas_grid = np.linspace(tasa_min, tasa_max, 1000)
    
    # Predecir la probabilidad de aceptación usando el modelo ajustado
    X_grid = sm.add_constant(tasas_grid)
    acceptance_probability = result.predict(X_grid)
    
    # Asegurar que las probabilidades están en el rango [0, 1]
    acceptance_probability = np.clip(acceptance_probability, 0, 1)
    
    # Calcular valores medios necesarios para el cálculo de revenue
    n = df_cluster['Plazo_Simulado'].mean()
    vp = df_cluster['Monto_Simulado'].mean()
    pnp = df_cluster['Probabilidad_No_Pago'].mean()
    data = {
        'Plazo_Simulado_medio': n, 
        'Monto_Simulado_medio': vp, 
        'Probabilidad_No_Pago_media': pnp
    }
    
    # Calcular el revenue potencial
    i = tasas_grid / 100  # Convertir a decimal
    one_plus_i_pow_n = np.power(1 + i, n)
    annuity_factor = (i * one_plus_i_pow_n) / (one_plus_i_pow_n - 1)
    revenue = (n * vp * annuity_factor) - vp
    potential_revenue = revenue * (1 - pnp)
    
    # Calcular el promedio de simulaciones por fecha
    df_cluster_simulaciones_1 = df_cluster[df_cluster['simulo'] == 1]
    num_dates = df_cluster_simulaciones_1['fecha'].nunique()
    total_simulaciones = df_cluster_simulaciones_1['simulo'].sum()
    simulaciones_medias = total_simulaciones / num_dates if num_dates else 0
    
    # Saltar el cluster si no hay simulaciones
    if simulaciones_medias == 0:
        continue
    
    # Calcular el revenue esperado
    expected_revenue = acceptance_probability * potential_revenue * simulaciones_medias
    
    # Encontrar la tasa que maximiza el revenue esperado
    idx_max = np.argmax(expected_revenue)
    max_price = tasas_grid[idx_max]
    max_expected_revenue = expected_revenue[idx_max]
    
    # Probabilidad de aceptación en la tasa óptima
    prob_aceptacion_optima = acceptance_probability[idx_max]
    
    # Número esperado de créditos aceptados
    num_creditos_aceptados = round(prob_aceptacion_optima * simulaciones_medias)
    
    # Número de clientes únicos en el cluster
    num_clients = df_cluster['rut'].nunique()
    
    # Imprimir resultados para cada cluster
    print(f'Cluster {cluster_num}:')
    print(f'- Precio Máx. Revenue Esperado = {max_price:.2f}%')
    print(f'- Revenue Esperado Máximo = {max_expected_revenue:,.2f}')
    print(f'- Número de clientes en el cluster = {num_clients}')
    print(f'- Número de simulaciones en el cluster = {simulaciones_medias:.2f}')
    print(f'- Probabilidad de aceptación en el precio óptimo = {prob_aceptacion_optima:.4f}')
    print(f'- Número esperado de créditos aceptados = {num_creditos_aceptados}')
    print(f'- Monto medio simulado = {data["Monto_Simulado_medio"]:,.2f}')
    print(f'- Plazo medio simulado = {data["Plazo_Simulado_medio"]:,.2f}')
    print(f'- Probabilidad de no pago media = {data["Probabilidad_No_Pago_media"]:.4f}\n')
    
    # Agregar resultados a las listas globales
    lista_clientes.append(num_clients)
    lista_revenue.append(max_expected_revenue)
    lista_creditos.append(num_creditos_aceptados)
    lista_simulaciones.append(simulaciones_medias)
    
    # Almacenar resultados por cluster en cluster_results
    cluster_results.append({
        'categoria_clusterizacion_numerica': cluster_num,
        'tasa_optima': max_price,
        'probabilidad_aceptacion_optima': prob_aceptacion_optima,
        'revenue_esperado_maximo': max_expected_revenue,
        'numero_clientes': num_clients,
        'numero_simulaciones_medias': simulaciones_medias,
        'numero_creditos_esperados': num_creditos_aceptados,
        'monto_medio_simulado': data["Monto_Simulado_medio"],
        'plazo_medio_simulado': data["Plazo_Simulado_medio"],
        'probabilidad_no_pago_media': data["Probabilidad_No_Pago_media"]
    })

# Imprimir resultados globales
total_revenue = sum(lista_revenue)
total_clientes = sum(lista_clientes)
total_simulaciones = sum(lista_simulaciones)
total_creditos = sum(lista_creditos)

print(f"El revenue total esperado es: {total_revenue:,.2f} con un total de {total_clientes} clientes, "
      f"{total_simulaciones:,.2f} simulaciones, y {total_creditos} créditos.")

# Crear un DataFrame a partir de cluster_results
df_cluster_results = pd.DataFrame(cluster_results)

# Incorporar los resultados por cluster de 'df_cluster_results' a 'df_estimar_elasticidad'
df_estimar_elasticidad = df_estimar_elasticidad.merge(
    df_cluster_results[['categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']],
    on='categoria_clusterizacion_numerica', 
    how='left'
)


Cluster 0:
- Precio Máx. Revenue Esperado = 2.50%
- Revenue Esperado Máximo = 743,866,611.30
- Número de clientes en el cluster = 39964
- Número de simulaciones en el cluster = 9176.47
- Probabilidad de aceptación en el precio óptimo = 0.4015
- Número esperado de créditos aceptados = 3684
- Monto medio simulado = 526,521.07
- Plazo medio simulado = 27.48
- Probabilidad de no pago media = 0.0281

Cluster 4:
- Precio Máx. Revenue Esperado = 1.00%
- Revenue Esperado Máximo = 1,384,106,953.63
- Número de clientes en el cluster = 49001
- Número de simulaciones en el cluster = 11468.29
- Probabilidad de aceptación en el precio óptimo = 0.7431
- Número esperado de créditos aceptados = 8522
- Monto medio simulado = 1,120,980.25
- Plazo medio simulado = 27.50
- Probabilidad de no pago media = 0.0262

Cluster 7:
- Precio Máx. Revenue Esperado = 1.00%
- Revenue Esperado Máximo = 41,886,293,892.75
- Número de clientes en el cluster = 82633
- Número de simulaciones en el cluster = 21163.65
- Probab

# 5. Estimacion de respuesta a tratamiento por cluster

In [22]:
# Unir los DataFrames 'df_tratamiento' y 'df_simulaciones_clientes' usando las columnas 'rut' y 'fecha' como claves
# La unión se realiza con 'how="left"', lo cual asegura que todos los registros de 'df_tratamiento' se conserven,
# incluyendo aquellos sin coincidencia en 'df_simulaciones_clientes'.
# Esta operación permite combinar la información de tratamiento con los datos de simulaciones de clientes.
df_simulaciones_info = pd.merge(df_tratamiento, df_simulaciones_clientes, on=['rut', 'fecha'], how='left')


In [23]:
# Usar operaciones de cadenas vectorizadas para crear la columna 'Tratamiento'
# Esta columna concatenará información sobre el ejecutivo asignado y el número de correos enviados.
# Se convierte 'asg_ejec' a string para poder concatenar, y 'n_correos' se convierte primero a entero y luego a string.
# El formato final es: "Ejecutivo=<valor_asg_ejec>, Correos=<valor_n_correos>"
df_simulaciones_info['Tratamiento'] = (
    'Ejecutivo=' + df_simulaciones_info['asg_ejec'].astype(str) +
    ', Correos=' + df_simulaciones_info['n_correos'].astype(int).astype(str)
)


In [24]:
# Extraer el mes y año de la columna 'fecha' y crear una nueva columna 'mes' en formato de periodo mensual
# Se convierte 'fecha' al formato datetime y luego se usa 'dt.to_period('M')' para obtener el mes/año.
df_simulaciones_info['mes'] = pd.to_datetime(df_simulaciones_info['fecha']).dt.to_period('M')

# Filtrar y mostrar las filas donde 'rut' es igual a 1
# Este filtro permite observar los registros específicos del cliente con 'rut' igual a 1, 
# lo cual es útil para verificar datos o analizar un cliente en particular.
df_simulaciones_info[df_simulaciones_info['rut'] == 1]


,Unnamed: 0_x,fecha,rut,n_correos,asg_ejec,Unnamed: 0_y,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Tratamiento,mes
0,0,2019-01-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-01
543651,0,2019-02-01,1,4,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=4",2019-02
1087302,0,2019-03-01,1,1,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=1",2019-03
1630953,0,2019-04-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-04
2174604,0,2019-05-01,1,0,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=0",2019-05
...,...,...,...,...,...,...,...,...,...,...,...
33162711,0,2024-02-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-02
33706362,0,2024-03-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-03
34250013,0,2024-04-01,1,2,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=2",2024-04
34793664,0,2024-05-01,1,2,1,7864785.0,267268.0,31.0,2.027869,"Ejecutivo=1, Correos=2",2024-05


In [25]:
# Crear una nueva columna 'simulo' para indicar si el cliente tiene un registro de simulación
# La columna 'Unnamed: 0_y' se utiliza para verificar si hay un valor no nulo, lo que implica que hay una simulación.
# 'notna()' devuelve True para valores no nulos y False para valores nulos; luego, 'astype(int)' convierte estos valores a 1 (True) o 0 (False).
df_simulaciones_info['simulo'] = df_simulaciones_info['Unnamed: 0_y'].notna().astype(int)

# Filtrar y mostrar las filas donde 'rut' es igual a 1
# Este filtro permite observar los registros específicos del cliente con 'rut' igual a 1, 
# útil para verificar si la columna 'simulo' refleja correctamente la presencia de simulaciones para este cliente.
df_simulaciones_info[df_simulaciones_info['rut'] == 1]


,Unnamed: 0_x,fecha,rut,n_correos,asg_ejec,Unnamed: 0_y,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Tratamiento,mes,simulo
0,0,2019-01-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-01,0
543651,0,2019-02-01,1,4,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=4",2019-02,0
1087302,0,2019-03-01,1,1,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=1",2019-03,0
1630953,0,2019-04-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-04,0
2174604,0,2019-05-01,1,0,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=0",2019-05,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33162711,0,2024-02-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-02,0
33706362,0,2024-03-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-03,0
34250013,0,2024-04-01,1,2,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=2",2024-04,0
34793664,0,2024-05-01,1,2,1,7864785.0,267268.0,31.0,2.027869,"Ejecutivo=1, Correos=2",2024-05,1


In [26]:
# Crear una copia del DataFrame 'df_estimar_elasticidad' con solo las columnas especificadas
# 'df1' contiene las columnas 'rut', 'categoria_clusterizacion_numerica', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
# Esta copia es útil para trabajar con los datos de elasticidad y clusterización sin modificar el DataFrame original.
df1 = df_estimar_elasticidad[['rut', 'categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']].copy()

# Crear una copia del DataFrame 'df_simulaciones_info' con solo las columnas especificadas
# 'df2' contiene las columnas 'rut', 'mes', 'Tratamiento' y 'simulo'.
# Esta copia es útil para trabajar con los datos de tratamiento y simulación en un conjunto de datos reducido.
df2 = df_simulaciones_info[['rut', 'mes', 'Tratamiento', 'simulo']].copy()


In [27]:
# Paso 1: Preparación de datos y mapeo de clusters
# Eliminar duplicados en 'df1' para tener un valor único de 'categoria_clusterizacion_numerica' por cada 'rut'.
df1_unique = df1.drop_duplicates(subset='rut')

# Crear un mapeo de 'rut' a 'categoria_clusterizacion_numerica' para asociar cada cliente a su cluster numérico.
rut_cluster_map = df1_unique.set_index('rut')['categoria_clusterizacion_numerica']

# Mapear la categoría de cluster a cada 'rut' en 'df2' usando el mapeo creado
df2['categoria_clusterizacion_numerica'] = df2['rut'].map(rut_cluster_map)

# Eliminar filas donde 'categoria_clusterizacion_numerica' es nulo, es decir, aquellos 'rut' sin mapeo de cluster.
df2 = df2.dropna(subset=['categoria_clusterizacion_numerica'])

# Conversión de tipos de datos
# Convertir 'categoria_clusterizacion_numerica' a entero para garantizar un tipo de dato consistente.
df2['categoria_clusterizacion_numerica'] = df2['categoria_clusterizacion_numerica'].astype(int)

# Convertir 'simulo' a numérico, reemplazando valores nulos por 0 y asegurando que sea un tipo de dato entero.
df2['simulo'] = pd.to_numeric(df2['simulo'], errors='coerce').fillna(0).astype(int)

# Convertir 'Tratamiento' a tipo de categoría para optimizar espacio y realizar operaciones categóricas.
df2['Tratamiento'] = df2['Tratamiento'].astype('category')

# Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
# Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número total de registros en cada combinación.
total_entries_per_cluster_treatment = df2.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

# Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
# Filtrar filas donde 'simulo' es 1 (clientes que realizaron una simulación)
df_simulations = df2[df2['simulo'] == 1]

# Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número de registros favorables (simulaciones).
favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

# Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
# Realizar un merge entre 'total_entries_per_cluster_treatment' y 'favorable_entries_per_cluster_treatment' en las columnas de cluster y tratamiento.
df_probabilities = total_entries_per_cluster_treatment.merge(
    favorable_entries_per_cluster_treatment,
    on=['categoria_clusterizacion_numerica', 'Tratamiento'],
    how='left'
)

# Llenar valores nulos en 'caso_favorable' con 0, asegurando que solo las columnas numéricas estén afectadas.
df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

# Asegurar que 'caso_total' sea de tipo entero para evitar inconsistencias en los conteos.
df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

# Calcular la probabilidad de simulación como el cociente entre 'caso_favorable' y 'caso_total'.
df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

# Organizar las columnas del DataFrame resultante para facilitar su análisis.
df_probabilities = df_probabilities[[
    'categoria_clusterizacion_numerica',
    'Tratamiento',
    'probabilidad_simular',
    'caso_favorable',
    'caso_total'
]]

# Mostrar el DataFrame resultante con la probabilidad de simulación calculada para cada combinación de cluster y tratamiento.
df_probabilities


,categoria_clusterizacion_numerica,Tratamiento,probabilidad_simular,caso_favorable,caso_total
0,0,"Ejecutivo=0, Correos=0",0.142091,30963,217909
1,0,"Ejecutivo=0, Correos=1",0.154878,36370,234830
2,0,"Ejecutivo=0, Correos=2",0.166170,30098,181128
3,0,"Ejecutivo=0, Correos=3",0.173276,78393,452418
4,0,"Ejecutivo=0, Correos=4",0.176019,127619,725028
...,...,...,...,...,...
91,11,"Ejecutivo=0, Correos=3",0.170259,71129,417770
92,11,"Ejecutivo=0, Correos=4",0.173859,116267,668742
93,11,"Ejecutivo=1, Correos=0",0.344585,43568,126436
94,11,"Ejecutivo=1, Correos=1",0.353329,48346,136830


In [45]:
# Visualizar el DataFrame 'df_estimar_elasticidad' para revisar su contenido antes de realizar cálculos adicionales
df_estimar_elasticidad

# Calcular el valor promedio de 'Monto_Simulado' para cada 'categoria_clusterizacion_numerica'
# Usamos 'groupby' para agrupar por 'categoria_clusterizacion_numerica' y 'transform("mean")' para calcular el promedio.
# Luego, 'transform' asigna este valor promedio a cada fila dentro de su grupo, creando una columna 'Monto_Simulado_mean' con estos promedios.
df_estimar_elasticidad['Monto_Simulado_mean'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Monto_Simulado'].transform('mean')

# Calcular el valor promedio de 'Plazo_Simulado' para cada 'categoria_clusterizacion_numerica'
# Similar al cálculo anterior, 'groupby' agrupa los datos por 'categoria_clusterizacion_numerica', y 'transform("mean")' calcula el promedio.
# Se asigna el promedio resultante a cada fila dentro del grupo en la nueva columna 'Plazo_Simulado_mean'.
df_estimar_elasticidad['Plazo_Simulado_mean'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform('mean')
df_estimar_elasticidad['Plazo_Simulado_min'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform('min')
df_estimar_elasticidad['Plazo_Simulado_max'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform('max')
df_estimar_elasticidad['Plazo_Simulado_mode'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform(lambda x: x.mode().iloc[0])


In [46]:
# Seleccionar solo las columnas necesarias del DataFrame 'df_estimar_elasticidad' para reducir su tamaño
# 'df_estimar_elasticidad_small' contiene las columnas esenciales para el análisis:
# 'categoria_clusterizacion_numerica', 'rut', 'tasa_optima', 'probabilidad_aceptacion_optima', 'Probabilidad_No_Pago',
# 'Monto_Simulado_mean', y 'Plazo_Simulado_mean'.
df_estimar_elasticidad_small = df_estimar_elasticidad[['categoria_clusterizacion_numerica', 'rut', 'tasa_optima', 'probabilidad_aceptacion_optima', 'Probabilidad_No_Pago', 
                                                       'Monto_Simulado_mean',
                                                       'Plazo_Simulado_mean', 'Plazo_Simulado_min', 'Plazo_Simulado_max', 'Plazo_Simulado_mode']]

# Seleccionar solo las columnas necesarias del DataFrame 'df_probabilities' para reducir su tamaño
# 'df_probabilities_small' contiene las columnas 'categoria_clusterizacion_numerica', 'probabilidad_simular', y 'Tratamiento'.
df_probabilities_small = df_probabilities[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']]

# Realizar un merge entre 'df_estimar_elasticidad_small' y 'df_probabilities_small' usando 'categoria_clusterizacion_numerica' como clave
# Esta unión ('how="left"') mantiene todas las filas de 'df_estimar_elasticidad_small' y añade la información de 'df_probabilities_small'
# cuando hay coincidencias en 'categoria_clusterizacion_numerica'. El resultado se guarda en 'df_asignacion_de_tratamientos'.
df_asignacion_de_tratamientos = pd.merge(df_estimar_elasticidad_small, df_probabilities_small, on='categoria_clusterizacion_numerica', how='left')


In [47]:
# Crear un nombre de carpeta con una marca de tiempo actual
# 'strftime' genera la fecha y hora actual en el formato "YYYYMMDD_HHMMSS".
# Esto se usa para crear una carpeta única 'folder_name' donde se guardarán los archivos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"cluster_data_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si ya existe.

# Guardar información de los clusters
# Seleccionar las columnas relevantes sobre cada cluster desde 'df_estimar_elasticidad_small' y eliminar duplicados.
# El DataFrame 'df_cluster_info' contiene datos únicos de cada cluster como el monto y plazo medio simulado, la probabilidad de aceptación óptima y la tasa óptima.
df_cluster_info = df_estimar_elasticidad_small[['categoria_clusterizacion_numerica', 'probabilidad_aceptacion_optima', 'tasa_optima',
                                                'Monto_Simulado_mean',
                                                'Plazo_Simulado_mean', 'Plazo_Simulado_min', 'Plazo_Simulado_max', 'Plazo_Simulado_mode']].drop_duplicates()
df_cluster_info.to_csv(f"{folder_name}/cluster_info.csv", index=False)

# Guardar las probabilidades y tratamiento
# Seleccionar columnas relevantes de 'df_probabilities_small' para almacenar la probabilidad de simulación y tratamiento asignado para cada cluster.
# 'df_probabilities_treatment' contiene esta información única por cada combinación de cluster y tratamiento.
df_probabilities_treatment = df_probabilities_small[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']].drop_duplicates()
df_probabilities_treatment.to_csv(f"{folder_name}/probabilities_treatment.csv", index=False)

# Guardar información del RUT
# Seleccionar columnas clave sobre cada cliente ('rut') desde 'df_estimar_elasticidad_small' y eliminar duplicados.
# 'df_rut_info' contiene el 'rut', la categoría de cluster y la probabilidad de no pago para cada cliente, sin registros duplicados.
df_rut_info = df_estimar_elasticidad_small[['rut', 'categoria_clusterizacion_numerica', 'Probabilidad_No_Pago']].drop_duplicates()
df_rut_info.to_csv(f"{folder_name}/rut_info.csv", index=False)


# 6. Modelo de asignacion

## Modelo de asignacion que itera por cliente (FLEXIBLE)

In [63]:
# Complete Optimized Code: Data Processing and Optimization

import pandas as pd
import os
import numpy as np
from gurobipy import Model, GRB, quicksum

# -------------------------------
# Data Processing and Preprocessing
# -------------------------------

tratamiento_map = {
    "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
}

# Parameters
costosms = 100
capacidad_ejecutivos = 205000

# Step 1: Load and map 'tratamiento_id' in the probabilities data
print("Loading and processing probabilities data...")
df_probabilities = df_probabilities_treatment
df_probabilities['tratamiento_id'] = df_probabilities['Tratamiento'].map(tratamiento_map)

# Step 2: Create 'tratamientos' list y merge con rut_info
print("Merging probabilities with rut_info...")
df_probabilities['tratamientos'] = df_probabilities[['probabilidad_simular', 'tratamiento_id']].values.tolist()
grouped_prob = df_probabilities.groupby('categoria_clusterizacion_numerica')['tratamientos'].apply(list).reset_index()

df_rut_info = df_rut_info
df_rut_info = df_rut_info.merge(grouped_prob, on='categoria_clusterizacion_numerica', how='left')
df_rut_info.drop(columns='tratamientos_y')
df_rut_info.rename(columns={'tratamientos_x': 'tratamientos'})

# Step 3: Merge rut_info with cluster_info
print("Merging rut_info with cluster_info...")
df_cluster_info = df_cluster_info
df_rut_info = df_rut_info.merge(df_cluster_info, on='categoria_clusterizacion_numerica', how='left')

# Step 4: Calculate 'RC'
print("Calculating RC...")
df_rut_info['tasa_optima'] /= 100
df_rut_info['RC'] = (
    (df_rut_info['Plazo_Simulado_mean'] * df_rut_info['Monto_Simulado_mean'] * df_rut_info['tasa_optima'] *
     ((1 + df_rut_info['tasa_optima']) ** df_rut_info['Plazo_Simulado_mean'])) /
    (((1 + df_rut_info['tasa_optima']) ** df_rut_info['Plazo_Simulado_mean']) - 1)
) - df_rut_info['Monto_Simulado_mean']

# -------------------------------
# Data Preparation for Optimization
# -------------------------------

# Convert 'tratamientos' to numpy array to improve indexing performance
profits = np.array([
    [
        (row['RC'] * (1 - row['Probabilidad_No_Pago']) * row['probabilidad_aceptacion_optima'] * row['tratamientos'][t][0]) - 
        (row['tratamientos'][t][1] * costosms)
        for t in range(len(row['tratamientos']))
    ]
    for _, row in df_rut_info.iterrows()
])

# Initialize model
model = Model("Maximizar_Ganancias")
model.ModelSense = GRB.MAXIMIZE

# Create decision variables and set objective
n_clients, n_treatments = profits.shape
variables = {}

for i in range(n_clients):
    variables[i] = {}
    for t in range(n_treatments):
        if profits[i, t] > 0:
            variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")

model.setObjective(
    quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
)

# Constraint: Each client receives exactly one treatment
for i in variables:
    model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")

# Capacity constraint for executives
model.addConstr(
    quicksum(variables[i][t] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
    name="CapacityConstraint"
)

# -------------------------------
# Cluster Flexibility: Allow Multiple Treatments per Cluster
# -------------------------------

print("Adding cluster flexibility constraints...")

# Definir el número máximo de tratamientos permitidos por clúster
max_tratamientos_por_cluster = 2  # Puedes ajustar este valor según tus necesidades

clusters = df_rut_info.groupby("categoria_clusterizacion_numerica").indices
for cluster_id, indices_cluster in clusters.items():
    indices_list = list(indices_cluster)
    
    # Crear variables binarias para tratamientos por clúster
    tratamientos_cluster = {}
    for t in range(n_treatments):
        tratamientos_cluster[t] = model.addVar(vtype=GRB.BINARY, name=f"y_{cluster_id}_{t}")
    
    # Vincular variables de tratamiento de cliente con tratamientos del clúster
    for i in indices_list:
        for t in variables[i]:
            # Si el tratamiento t es asignado al cliente i, entonces el tratamiento t debe estar asignado al clúster
            model.addConstr(variables[i][t] <= tratamientos_cluster[t], name=f"LinkCluster_{cluster_id}_Client_{i}_Treatment_{t}")
    
    # Limitar el número de tratamientos por clúster
    model.addConstr(
        quicksum(tratamientos_cluster[t] for t in tratamientos_cluster) <= max_tratamientos_por_cluster,
        name=f"MaxTratamientosCluster_{cluster_id}"
    )

# -------------------------------
# Optimize the model
# -------------------------------

# Optimize the model
model.optimize()

# Check if the optimization was successful
if model.Status == GRB.OPTIMAL:
    # -------------------------------
    # Extracting and Displaying Results
    # -------------------------------
    print("Extracting results...")

    # Asignar tratamientos por clúster basados en los resultados de la optimización
    resultados_por_cluster = {}
    for cluster_id, indices_cluster in clusters.items():
        tratamientos_asignados = []
        for t in range(n_treatments):
            tratamiento_var = model.getVarByName(f"y_{cluster_id}_{t}")
            if tratamiento_var.X > 0.5:
                tratamientos_asignados.append(t)
        resultados_por_cluster[cluster_id] = tratamientos_asignados

    # Calcular ganancias totales
    ganancias_totales = model.ObjVal

    # Mostrar resultados
    print("\nTratamientos asignados por cluster:")
    for cluster_id, tratamientos in resultados_por_cluster.items():
        tratamientos_str = ', '.join([str(t + 1) for t in tratamientos])
        print(f"Cluster {cluster_id}: Tratamientos {tratamientos_str}")

    print(f"\nGanancias totales: {ganancias_totales:.2f}")

    # -------------------------------
    # Calculating Executive Usage
    # -------------------------------
    
    # Contar el número de ejecutivos usados para tratamientos 5, 6 y 7
    executives_used = sum(
        1 for i in variables for t in variables[i] if t in [5, 6, 7] and variables[i][t].X > 0.5
    )
    executives_remaining = capacidad_ejecutivos - executives_used

    # Mostrar resumen del uso de ejecutivos
    print(f"\nExecutives used: {executives_used}")
    print(f"Executives remaining: {executives_remaining}")

else:
    print("Optimization did not reach an optimal solution.")
print("Optimization complete.")


Loading and processing probabilities data...
Merging probabilities with rut_info...
Merging rut_info with cluster_info...
Calculating RC...
Adding cluster flexibility constraints...
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4892872 rows, 4349304 columns and 14678673 nonzeros
Model fingerprint: 0xf754c549
Variable types: 0 continuous, 4349304 integer (4349304 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 8e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 0 rows and 0 columns (presolve time = 11s) ...
Presolve removed 0 rows and 0 columns (presolve time = 16s) ...
Presolve removed 0 rows and 0 columns (presolve ti

### Resultados

In [108]:
# Suponiendo que 'variables' es un diccionario de diccionarios donde variables[i][t] es una variable Gurobi
# y que 'df_rut_info' contiene las columnas 'rut' y 'categoria_clusterizacion_numerica'

# Extraer las columnas necesarias como arrays para acceso más rápido
ruts = df_rut_info['rut'].values
clusters = df_rut_info['categoria_clusterizacion_numerica'].values

# Crear una lista de tuplas con las asignaciones utilizando list comprehension
asignaciones = [
    (ruts[i], t + 1, clusters[i])
    for i, t_dict in variables.items()
    for t, var in t_dict.items()
    if var.X > 0.5
]

# Convertir la lista de tuplas en un DataFrame
df_asignaciones = pd.DataFrame(asignaciones, columns=['rut', 'assigned_treatment', 'cluster'])

df_asignaciones


,rut,assigned_treatment,cluster
0,1,5,0
1,2,5,4
2,3,8,7
3,4,5,0
4,5,5,8
...,...,...,...
543646,543647,5,4
543647,543648,5,5
543648,543649,5,9
543649,543650,5,9


In [109]:
# Contar asignaciones por clúster y tratamiento
tratamiento_cluster_counts = df_asignaciones.groupby(['cluster', 'assigned_treatment']).size().unstack(fill_value=0)

# Mostrar el resultado
print("\nConteo de asignaciones por clúster y tratamiento:")
print(tratamiento_cluster_counts)


Conteo de asignaciones por clúster y tratamiento:
assigned_treatment      5      8
cluster                         
0                   40091      0
1                   52062      0
2                   12994      0
3                       0  19546
4                   49057      0
5                   11961  61545
6                   27065      0
7                       0  82671
8                   54307      0
9                   69628      0
10                  21486      0
11                      0  41238


In [114]:
df_asignaciones

,rut,assigned_treatment,cluster
0,1,5,0
1,2,5,4
2,3,8,7
3,4,5,0
4,5,5,8
...,...,...,...
543646,543647,5,4
543647,543648,5,5
543648,543649,5,9
543649,543650,5,9


In [118]:
print(df_asignaciones['cluster'].duplicated().sum())  # Count duplicates in 'cluster'
print(df_rut_info['categoria_clusterizacion_numerica'].duplicated().sum())  # Count duplicates in 'categoria_clusterizacion_numerica'


543639
543639


In [139]:
df_rut_merge = df_rut_info[['rut', 'tratamientos', 'categoria_clusterizacion_numerica', 'Probabilidad_No_Pago', 'probabilidad_aceptacion_optima',	'tasa_optima',	'Monto_Simulado_mean',	'Plazo_Simulado_mean',	'Plazo_Simulado_min',	'Plazo_Simulado_max',	'Plazo_Simulado_mode']]

In [ ]:
df_rut_merge.rename(columns = {'categoria_clusterizacion_numerica': 'cluster'})

In [141]:
df_assigned = pd.merge(
    df_asignaciones,
    df_rut_merge,
    how='left'
)


In [ ]:
df_assigned.drop(columns = {'categoria_clusterizacion_numerica'})

In [144]:
# Calcular la probabilidad de simulación para el tratamiento asignado en cada fila de 'df_assigned'
# Se usa 'apply' con una función lambda para extraer la probabilidad de simulación correspondiente al tratamiento asignado.
# 'row['tratamientos']' es una lista de opciones de tratamiento y 'row['assigned_treatment'] - 1' indica la posición del tratamiento.
# Primero, se verifica que 'tratamientos' sea una lista y que el índice calculado esté dentro del rango.
# Si estas condiciones se cumplen, se extrae la probabilidad de simulación; en caso contrario, se asigna None.
df_assigned['probabilidad_de_simular'] = df_assigned.apply(
    lambda row: row['tratamientos'][row['assigned_treatment'] - 1][0] 
                if isinstance(row['tratamientos'], list) and (0 <= row['assigned_treatment'] - 1 < len(row['tratamientos'])) 
                else None,
    axis=1
)


In [146]:
import os
from datetime import datetime

# Crear una carpeta con un nombre basado en la fecha y hora actual
# 'strftime' genera un timestamp en el formato "YYYYMMDD_HHMMSS" para asegurar nombres de carpeta únicos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"assigned_treatments/assignation_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si la carpeta ya existe.

# Definir la ruta del archivo CSV dentro de la nueva carpeta
output_path = os.path.join(folder_name, 'assigned_treatments.csv')

# Guardar el DataFrame 'df_assigned' con las columnas seleccionadas en un archivo CSV
# Se incluyen las columnas clave: 'rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment',
# 'probabilidad_de_simular', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
df_assigned[['rut', 'cluster', 'Probabilidad_No_Pago', 'assigned_treatment', 'probabilidad_de_simular', 'tasa_optima', 'probabilidad_aceptacion_optima']].to_csv(output_path, index=False)

# Imprimir mensaje de confirmación con la ubicación del archivo CSV guardado
print(f"CSV file saved in folder: {output_path}")


CSV file saved in folder: assigned_treatments/assignation_20241117_140420\assigned_treatments.csv


## Modelo de asignacion que itera por cluster

In [148]:
# -------------------------------
# Procesamiento y preprocesamiento de datos
# -------------------------------

# Definir la carpeta base y el mapeo de tratamientos
base_folder = "cluster_data_20241106_020021/"  # Carpeta donde se encuentran los archivos de datos
tratamiento_map = {  # Mapeo de los tratamientos específicos a identificadores numéricos
    "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
}

# Definir las rutas de los archivos
probabilities_file = os.path.join(base_folder, "probabilities_treatment.csv")
rut_info_file = os.path.join(base_folder, "rut_info.csv")
cluster_info_file = os.path.join(base_folder, "cluster_info.csv")

# Parámetros
costosms = 100  # Costo de cada mensaje SMS
capacidad_ejecutivos = 205000  # Capacidad máxima en términos de tiempo de los ejecutivos

# Paso 1: Cargar y mapear 'tratamiento_id' en los datos de probabilidades
print("Loading and processing probabilities data...")
df_probabilities = df_probabilities_treatment
df_probabilities['tratamiento_id'] = df_probabilities['Tratamiento'].map(tratamiento_map)

# Paso 2: Crear lista de tratamientos y combinar con rut_info
print("Merging probabilities with rut_info...")
df_probabilities['tratamientos'] = df_probabilities[['probabilidad_simular', 'tratamiento_id']].values.tolist()
grouped_prob = df_probabilities.groupby('categoria_clusterizacion_numerica')['tratamientos'].apply(list).reset_index()

df_rut_info = df_rut_info
df_rut_info = df_rut_info.merge(grouped_prob, on='categoria_clusterizacion_numerica', how='left')

# Paso 3: Combinar rut_info con cluster_info
print("Merging rut_info with cluster_info...")
df_cluster_info = df_cluster_info
df_rut_info = df_rut_info.merge(df_cluster_info, on='categoria_clusterizacion_numerica', how='left')

# Paso 3.5: Agrupar información por cluster en 'rut_info'
# Agrupar por 'categoria_clusterizacion_numerica' y agregar según lo especificado
df_grouped = df_rut_info.groupby('categoria_clusterizacion_numerica').agg({
    'Probabilidad_No_Pago': 'mean',  # Promedio de probabilidad de no pago
    'tratamientos': lambda x: list(x),  # Lista de opciones de tratamiento únicas en cada cluster
    'Monto_Simulado_mean': 'mean',
    'Plazo_Simulado_mean': 'mean',
    'probabilidad_aceptacion_optima': 'mean',
    'tasa_optima': 'mean',
    'rut': 'count'  # Conteo del número de clientes ('rut') en cada cluster
}).rename(columns={'rut': 'n_clientes'}).reset_index()

# Paso 4: Calcular 'RC' (Revenue calculado)
print("Calculating RC...")
df_grouped['tasa_optima'] /= 100  # Convertir tasa óptima a decimal
df_grouped['RC'] = (
    (df_grouped['Plazo_Simulado_mean'] * df_grouped['Monto_Simulado_mean'] * df_grouped['tasa_optima'] *
     ((1 + df_grouped['tasa_optima']) ** df_grouped['Plazo_Simulado_mean'])) /
    (((1 + df_grouped['tasa_optima']) ** df_grouped['Plazo_Simulado_mean']) - 1)
) - df_grouped['Monto_Simulado_mean']

# -------------------------------
# Preparación de datos para optimización
# -------------------------------

# Convertir 'tratamientos' a un arreglo de numpy para mejorar la indexación
# Desarrollar y preparar 'tratamientos' para indexación adecuada
profits = np.array([
    [
        row['n_clientes'] * (row['RC'] * (1 - row['Probabilidad_No_Pago']) * row['probabilidad_aceptacion_optima'] * row['tratamientos'][0][t][0]) - 
        (row['tratamientos'][0][t][1] * costosms)
        for t in range(8)
    ]
    for _, row in df_grouped.iterrows()
])

# Inicializar el modelo de optimización
model = Model("Maximizar_Ganancias")
model.ModelSense = GRB.MAXIMIZE

# Crear variables de decisión y definir el objetivo
n_clients, n_treatments = profits.shape
variables = {}

for i in range(n_clients):
    variables[i] = {}
    for t in range(n_treatments):
        if profits[i, t] > 0:
            variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")

model.setObjective(
    quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
)

# Restricción: Cada cliente recibe exactamente un tratamiento
for i in variables:
    model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")

# Restricción de capacidad para ejecutivos
model.addConstr(
    quicksum(variables[i][t] * df_grouped.loc[i, 'n_clientes'] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
    name="CapacityConstraint"
)

# Consistencia de cluster: los clientes dentro del mismo cluster deben recibir el mismo tratamiento
clusters = df_grouped.groupby("categoria_clusterizacion_numerica").indices
for cluster_id, indices_cluster in clusters.items():
    indices_list = list(indices_cluster)
    leader_index = indices_list[0]
    for t in variables[leader_index]:
        leader_var = variables[leader_index][t]
        for i in indices_list[1:]:
            if t in variables[i]:
                model.addConstr(variables[i][t] == leader_var, name=f"ClusterConsistency_{cluster_id}_{t}")

# Optimizar el modelo
model.optimize()

# Verificar si la optimización fue exitosa
if model.Status == GRB.OPTIMAL:
    # -------------------------------
    # Extracción y visualización de resultados
    # -------------------------------

    print("Extracting results...")

    # Asignar tratamientos por cluster basado en los resultados de la optimización
    resultados_por_cluster = {}
    for cluster_id, indices_cluster in clusters.items():
        leader_index = list(indices_cluster)[0]
        for t in variables[leader_index]:
            if variables[leader_index][t].X > 0.5:
                resultados_por_cluster[cluster_id] = t + 1
                break

    # Calcular las ganancias totales
    ganancias_totales = model.ObjVal

    # Mostrar resultados
    print("\nTratamientos asignados por cluster:")
    for cluster_id, tratamiento in resultados_por_cluster.items():
        print(f"Cluster {cluster_id}: Tratamiento {tratamiento}")

    print(f"\nGanancias totales: {ganancias_totales:.2f}")

    # Calcular el número de ejecutivos usados y restantes
    executives_used = sum(
        df_grouped.loc[i, 'n_clientes'] for i in variables for t in variables[i]
        if t in [5, 6, 7] and variables[i][t].X > 0.5
    )
    executives_remaining = capacidad_ejecutivos - executives_used

    # Mostrar resumen de uso de ejecutivos
    print(f"\nExecutives used: {executives_used}")
    print(f"Executives remaining: {executives_remaining}")
else:
    print("Optimization did not reach an optimal solution.")
print("Optimization complete.")


Loading and processing probabilities data...
Merging probabilities with rut_info...
Merging rut_info with cluster_info...
Calculating RC...
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 96 columns and 132 nonzeros
Model fingerprint: 0x0e4446d3
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+04]
  Objective range  [2e+08, 6e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 6.275927e+10
Presolve removed 12 rows and 84 columns
Presolve time: 0.01s
Presolved: 1 rows, 12 columns, 12 nonzeros
Found heuristic solution: objective 9.950115e+1

### Resultados

In [39]:
# Contar cuántas veces se asigna cada tratamiento en los resultados por cluster
# Se utiliza un diccionario 'Counter' para contar las ocurrencias de cada tratamiento asignado en 'resultados_por_cluster'
for cluster_id, tratamiento in resultados_por_cluster.items():
    treatment_counts = Counter(resultados_por_cluster.values())

# Imprimir el conteo de asignaciones para cada tratamiento
# Se recorre 'treatment_counts' para mostrar cuántas veces se asignó cada tratamiento.
# 'treatment + 1' se utiliza para mostrar el número de tratamiento en base 1, haciendo el resultado más legible.
for treatment, count in treatment_counts.items():
    print(f"Treatment {treatment}: {count} times")


Treatment 5: 9 times
Treatment 8: 3 times


In [40]:
# Convertir el diccionario 'resultados_por_cluster' a un DataFrame
# El diccionario 'resultados_por_cluster' contiene el ID del cluster y el tratamiento asignado a cada uno.
# Se convierte a un DataFrame donde la primera columna es 'cluster' y la segunda 'assigned_treatment'.
df_resultados_por_cluster = pd.DataFrame(list(resultados_por_cluster.items()), columns=["cluster", "assigned_treatment"])

# Mostrar el DataFrame resultante al usuario
df_resultados_por_cluster


,cluster,assigned_treatment
0,0,5
1,1,5
2,2,5
3,3,5
4,4,5
5,5,8
6,6,5
7,7,8
8,8,5
9,9,5


In [41]:
# Primer merge con 'df_rut_info'
# Realizar una unión ('merge') entre 'df_resultados_por_cluster' y 'df_rut_info' usando 'cluster' en el primer DataFrame
# y 'categoria_clusterizacion_numerica' en el segundo como claves de unión.
# Esta unión permite agregar información de clientes a cada cluster con su tratamiento asignado.
df_assigned = pd.merge(df_resultados_por_cluster, df_rut_info, left_on='cluster', right_on='categoria_clusterizacion_numerica', how='left')

# Segundo merge con 'df_grouped' para agregar la columna 'RC'
# Realizar una unión entre 'df_assigned' y 'df_grouped' para incorporar la columna 'RC' (Revenue Calculado)
# Usamos 'categoria_clusterizacion_numerica' como clave de unión para añadir la información de revenue calculado a cada cluster.
df_assigned = pd.merge(df_assigned, df_grouped[['categoria_clusterizacion_numerica', 'RC']], on='categoria_clusterizacion_numerica', how='left')


In [42]:
# Calcular la probabilidad de simulación para el tratamiento asignado en cada fila de 'df_assigned'
# Se usa 'apply' con una función lambda para extraer la probabilidad de simulación correspondiente al tratamiento asignado.
# 'row['tratamientos']' es una lista de opciones de tratamiento y 'row['assigned_treatment'] - 1' indica la posición del tratamiento.
# Primero, se verifica que 'tratamientos' sea una lista y que el índice calculado esté dentro del rango.
# Si estas condiciones se cumplen, se extrae la probabilidad de simulación; en caso contrario, se asigna None.
df_assigned['probabilidad_de_simular'] = df_assigned.apply(
    lambda row: row['tratamientos'][row['assigned_treatment'] - 1][0] 
                if isinstance(row['tratamientos'], list) and (0 <= row['assigned_treatment'] - 1 < len(row['tratamientos'])) 
                else None,
    axis=1
)


In [44]:
import os
from datetime import datetime

# Crear una carpeta con un nombre basado en la fecha y hora actual
# 'strftime' genera un timestamp en el formato "YYYYMMDD_HHMMSS" para asegurar nombres de carpeta únicos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"assigned_treatments/assignation_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si la carpeta ya existe.

# Definir la ruta del archivo CSV dentro de la nueva carpeta
output_path = os.path.join(folder_name, 'assigned_treatments.csv')

# Guardar el DataFrame 'df_assigned' con las columnas seleccionadas en un archivo CSV
# Se incluyen las columnas clave: 'rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment',
# 'probabilidad_de_simular', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
df_assigned[['rut', 'cluster', 'Probabilidad_No_Pago', 'assigned_treatment', 'probabilidad_de_simular', 'tasa_optima', 'probabilidad_aceptacion_optima']].to_csv(output_path, index=False)

# Imprimir mensaje de confirmación con la ubicación del archivo CSV guardado
print(f"CSV file saved in folder: {output_path}")


CSV file saved in folder: assigned_treatments/assignation_20241117_123829\assigned_treatments.csv
